In [1]:
import numpy as np

from sr_gp_fs_methods import gp, gpshap, gpmi, new_method
from datasets import load_datasets
from pysr_utils import results_to_dataframe, plot_results

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

sns.set_theme()
warnings.filterwarnings("ignore", category=UserWarning, module="pysr")
warnings.filterwarnings("ignore", category=RuntimeWarning)

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
def rmse_loss(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    return rmse

def nrmse_loss(y_true, y_pred):
    rmse = rmse_loss(y_true, y_pred)
    return rmse / (y_true.max() - y_true.min())

In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd



In [51]:
dataset.data.original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Columns: 125 entries, pop to violentPerPop
dtypes: float64(98), int64(27)
memory usage: 311.7 KB


In [1]:
a = {"A": 3, "B": 4, "C": 5, "D":1, "E": 2}

In [48]:
target_columns

['murders',
 'murdPerPop',
 'rapes',
 'rapesPerPop',
 'robberies',
 'robbbPerPop',
 'assaults',
 'assaultPerPop',
 'burglaries',
 'burglPerPop',
 'larcenies',
 'larcPerPop',
 'autoTheft',
 'autoTheftPerPop',
 'arsons',
 'arsonsPerPop',
 'nonViolPerPop']

In [20]:
dataset.data.features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Columns: 125 entries, State to policBudgetPerPop
dtypes: float64(97), int64(27), object(1)
memory usage: 2.1+ MB
